In [1]:
from ultralytics import YOLO
from ultralytics.yolo.utils import coco80_to_coco91_class

YOLO.convert_coco("train_dataset/instances_default.json", "output/yolo_format", use_segments=True)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/Users/sonny/Library/Application Support/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
